### Model Training

In [1]:
from src.myscripts.model import Conv1DClassifier
import os
import torch
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import config
from src.myscripts.train import ModelTrainer

path=config.DATA_SPLIT_SAVE_DIR_PATH
x_train= np.load(os.path.join(path, "x_train.npy"), allow_pickle=True)
y_train= np.load(os.path.join(path, "y_train.npy"), allow_pickle=True)
x_val= np.load(os.path.join(path, "x_val.npy"), allow_pickle=True)
y_val= np.load(os.path.join(path, "y_val.npy"), allow_pickle=True)
x_train = x_train.astype(np.float32)
x_val = x_val.astype(np.float32)
y_train = y_train.astype("long")
y_val = y_val.astype("long")

x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
x_val_tensor = torch.tensor(x_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
val_dataset = TensorDataset(x_val_tensor, y_val_tensor)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_shape = x_train.shape[1:]
model = Conv1DClassifier(num_classes=43, input_shape=input_shape)




C:\Users\Bcom_\Documents\Projekty\Rozpoznawanie_dzwiekow_gitarowych\data\prepared_data


In [2]:
print(torch.cuda.is_available())

True


In [3]:
model_trainer = ModelTrainer(model, device)
model_trainer.train_model(train_dataset, val_dataset)
# Ładujemy najlepszy model
model = model_trainer.get_trained_model()
train_losses, val_losses = model_trainer.get_loss_data()

Epoch 1/10, Loss: 3.6969, Accuracy: 0.0867
Validation Loss: 2.5143, Validation Accuracy: 0.5690
Epoch 2/10, Loss: 2.4977, Accuracy: 0.3714
Validation Loss: 1.2778, Validation Accuracy: 0.8103
Epoch 3/10, Loss: 1.6602, Accuracy: 0.5833
Validation Loss: 0.7009, Validation Accuracy: 0.9241
Epoch 4/10, Loss: 1.0804, Accuracy: 0.7251
Validation Loss: 0.4367, Validation Accuracy: 0.9517
Epoch 5/10, Loss: 0.6611, Accuracy: 0.8069
Validation Loss: 0.2667, Validation Accuracy: 0.9552
Epoch 6/10, Loss: 0.4572, Accuracy: 0.8581
Validation Loss: 0.3627, Validation Accuracy: 0.9690
Validation loss did not improve for 1 epochs.
Epoch 7/10, Loss: 0.4306, Accuracy: 0.8778
Validation Loss: 0.1883, Validation Accuracy: 0.9828
Epoch 8/10, Loss: 0.3164, Accuracy: 0.9044
Validation Loss: 0.2734, Validation Accuracy: 0.9690
Validation loss did not improve for 1 epochs.
Epoch 9/10, Loss: 0.2896, Accuracy: 0.9261
Validation Loss: 0.1615, Validation Accuracy: 0.9724
Epoch 10/10, Loss: 0.2664, Accuracy: 0.9310


In [4]:
torch.save(model.state_dict(),f"{os.path.join(config.MODEL_DIR_PATH, type(model).__name__)}.pth")

In [5]:
np.save(f"{os.path.join(config.MODEL_HISTORY_DIR_PATH, type(model).__name__)}_val_losses.npy",val_losses)
np.save(f"{os.path.join(config.MODEL_HISTORY_DIR_PATH, type(model).__name__)}_train_losses.npy",train_losses)